# Hello and welcome to the notebook
## ANother heading
### A Third HEading

In [1]:
import openmc
import openmc.deplete
from simple_openmc_depletion import create_openmc_model
from extract_simple_depletion_results import plot_results

## Step 1: Setup the Transport Operator

In order to run a depletion simulation using OpenMC, it is necessary to initialize an object called an operator, which is used to run a transport simulation at a particular point in time when performig a depletion simulation. THis operator object can be though of as a wrapper around an OpenMC model object which allows it to be passed to an integrator object, which is then used to perform the depletion simulation. To do this we first need an OpenMC model which is properly configured. I have placed all of the OpenMC model setup code into a separate python file and placed it all into a function which allows me to expose the parameters that I want to adjust about the model and easily create many versions. This is the "create_openmc_model" function which takes in a percent of the blanket to be fertile material, as well as a DAGMC model. This is a little bit tricky because not any DAGMC model can be used, it has to be one which has the correct components with the correct names which correspond to the materials specified in the "simple_openmc_depletion.py" file. Nonetheless this was done to allow for multiple variations on the same model to be interchanged easily as tweaks to the geometry are made.